<a href="https://colab.research.google.com/github/JoaoEmanuel14/ufs-ia-trabalho-a2/blob/main/IA_Trabalho_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando Modelos Multimodais: Como Texto, Áudio e Imagem se Combinam em IA

## 1. Instalação de bibliotecas, imports e chaves das APIs

In [1]:
# Instalar bibliotecas necessárias para texto
!pip install openai google-generativeai -q

import openai
import google.generativeai as genai
from google.colab import userdata  # Para acessar as chaves armazenadas no Secrets

# Instalar bibliotecas necessárias para imagem
import torch
from PIL import Image
from IPython.display import display

# Instalar bibliotecas necessárias para áudio
!pip install openai vosk gtts -q
!pip install pydub

import wave
import json
import io
import os
import base64
import pydub
import urllib.request
import zipfile
from gtts import gTTS
from glob import glob
from vosk import Model, KaldiRecognizer
from IPython.display import Audio
from pydub import AudioSegment

# Pegar conteúdo do Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configurar chaves de API usando Secrets do Colab
try:
  openai_api_key = userdata.get('OPENAI_API_KEY')  # Chave da OpenAI
  google_api_key = userdata.get('GOOGLE_API_KEY')  # Chave do Google
  genai.configure(api_key=google_api_key)
except Exception as e:
  print("Erro ao carregar as chaves de API. Verifique se as chaves estão configuradas corretamente no Secrets.")
  raise e

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 112.5 MB/s eta 0:00:00
Mounted at /content/drive


## 2. Configuração dos clientes OpenAI e Google

### 2.1. Funções que processam áudio e imagem, gerando texto

In [2]:
# # Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)

# Função para chamada ao GPT-4o com imagem e texto
def gerar_texto_openai(imagem_path, texto):
  try:
    with open(imagem_path, "rb") as image_file:
      # Utilizando a biblioteca base64 para fazer a leitura da imagem
      imagem_base = base64.b64encode(image_file.read()).decode("utf-8")
      # É necessário transformar a imagem em URL para que o ChatGPT possa lê-la
      imagem_url = f"data:image/jpeg;base64,{imagem_base}"

      # Completion para o prompt
      response = client.chat.completions.create(
          model="gpt-4o",
          # Montando a mensagem
          messages=[
              {"role": "user", "content": [
                {"type": "text", "text": texto},
                {"type": "image_url", "image_url": {"url": imagem_url}}
              ]}
          ]
      )
      return response.choices[0].message.content

  except Exception as e:
    return f"Erro ao gerar texto com OpenAI: {str(e)}"

# Função para chamada ao Gemini com imagem e texto
def gerar_texto_google(imagem_path, texto):
  try:
    model = genai.GenerativeModel("gemini-1.5-pro")
    imagem = Image.open(imagem_path)
    # Completion para o prompt
    response = model.generate_content([texto, imagem])
    return response.text

  except Exception as e:
    return f"Erro ao gerar texto com Google Gemini: {str(e)}"

### 2.2. Text-To-Speech (TTS): funções de conversão $texto \implies áudio$

In [3]:
#Função para conversão de texto em fala (TTS) com gTTS
def tts_gtts(texto):
  try:
    # Criar o objeto gTTS
    tts = gTTS(texto, lang="pt")  # Define o idioma como pt-BR
    # Salvar o áudio em um arquivo MP3
    tts.save("gtts_audio.mp3")
    print("Áudio gerado e salvo como 'gtts_audio.mp3'.")
    return "gtts_audio.mp3"

  except Exception as e:
    print(f"Erro ao gerar áudio com gTTS: {str(e)}")
    return None

# Configurar o cliente da OpenAI
client = openai.OpenAI(api_key=openai_api_key)

# Função para conversão de texto em fala (TTS) com OpenAI
def tts_openai(texto):
  try:
    # Completion para o prompt
    response = client.audio.speech.create(
        model="tts-1",
        input=texto,
        voice="alloy"
    )
    return response.content  # Retorna o conteúdo de áudio diretamente

  except Exception as e:
    print(f"Erro ao gerar áudio com OpenAI TTS: {str(e)}")
    return None

### 2.3. Função auxiliar que converte áudio do formato `mp3` para `wav`

In [4]:
# Função que converte mp3 para wav
def converter_mp3_para_wav(mp3_path, wav_path="audio.wav"):
  try:
    audio = AudioSegment.from_mp3(mp3_path)
    # Colocando o áudio no padrao PCM 16kHz mono, 16-bit
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    # Exportando o áudio no formato correto
    audio.export(wav_path, format="wav")
    print(f"Conversão concluída: '{wav_path}'")
    return wav_path

  except Exception as e:
    print(f"Erro na conversão de MP3 para WAV: {e}")
    return None

### 2.4. Função para transcrição de áudio usando Vosk

In [5]:
# Transcrever áudio usando Vosk
def transcrever_audio(caminho_audio):
  try:
    # Verifica se o modelo de reconhecimento de voz já foi baixado
    if not os.path.exists("vosk-model-small-pt-0.3"):
      print("Baixando o modelo...")
      # Faz o download do modelo de voz em português
      urllib.request.urlretrieve(
          "https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip",
          "vosk-model-small-pt-0.3.zip"
      )
      # Extrai os arquivos do modelo compactado
      with zipfile.ZipFile("vosk-model-small-pt-0.3.zip", "r") as zip_ref:
        zip_ref.extractall(".")

    wf = wave.open(caminho_audio, "rb")

    # Verifica se o áudio está no formato correto (mono, 16-bit, 16kHz)
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
      print("Formato de áudio inválido para Vosk. Esperado: WAV 16kHz mono PCM.")
      return None

    # Carrega o modelo Vosk
    model = Model("vosk-model-small-pt-0.3")

    # Inicializa o reconhecedor com o modelo e a taxa de amostragem do áudio
    rec = KaldiRecognizer(model, wf.getframerate())
    results = []

    # Lê o áudio em blocos e processa com o reconhecedor
    while True:
      data = wf.readframes(4000) # Lê 4000 frames de cada vez
      if len(data) == 0:
        break
      # Se a transcrição parcial for feita
      if rec.AcceptWaveform(data):
        # Converte o resultado para dicionário
        result = json.loads(rec.Result())
        # Adiciona o texto transcrito à lista
        results.append(result.get("text", ""))

    # Adiciona a transcrição final (último trecho processado)
    final = json.loads(rec.FinalResult())
    results.append(final.get("text", ""))

    # Junta todos os trechos de texto em uma única string
    texto = " ".join(results).strip()

    # Retorna o texto final, caso este nao esteja vazio
    if texto:
      return texto
    # Se estiver vazio, é retornado None
    else:
      return None

  except Exception as e:
    print("Erro na transcrição de áudio:", e)
    return None

### 2.5. Manipulação do arquivo que será usado para a construção da narrativa


In [6]:
# Função responsável por gerar os a tupla de pares (imagem, áudio)
def gerar_pares_imagem_audio(pasta_imagens, pasta_audios_mp3):
  pares_imagem_audio = []

  # Ordena os arquivos para garantir a correspondência sequencial
  imagens = sorted(glob(os.path.join(pasta_imagens, "*.jpg")))
  audios_mp3 = sorted(glob(os.path.join(pasta_audios_mp3, "*.mp3")))

  # O processo é interrompido caso o tamanho da pasta de imagens e de áudios for diferente
  if len(imagens) != len(audios_mp3):
    print("Quantidade de imagens e áudios não bate!")
    return []

  # Gera os pares após converter o áudio para o formato correto
  for idx, imagem in enumerate(imagens):
    audio_mp3 = audios_mp3[idx]

    wav_path = f"audio_convertido_{idx + 1}.wav"
    # Converte os áudios de .mp3 para .wav
    audio_wav_convertido = converter_mp3_para_wav(audio_mp3, wav_path)

    if audio_wav_convertido:
      pares_imagem_audio.append((imagem, audio_wav_convertido))
    else:
      print(f"[{idx + 1}] Erro ao converter áudio {audio_mp3}.")

  return pares_imagem_audio

## 3. Entrada Multimodal

Fornecer uma imagem e um áudio descrevendo a cena, e pedir ao modelo para gerar um resumo textual.

In [7]:
# Função para limitar a entrada de texto transcrita
def limitar_texto(texto, limite=16000):
  if len(texto) > limite:
    return texto[:limite] + "..."

  return texto

# Entrada multimodal
def entrada_multimodal(caminho_imagem, caminho_audio_mp3):
  # Conversão do áudio para o formato correto
  caminho_wav = converter_mp3_para_wav(caminho_audio_mp3)
  if not caminho_wav:
    print("Erro ao converter áudio")
    return

  # Gera o texto transcrito a partir do áudio
  texto_transcrito = limitar_texto(transcrever_audio(caminho_wav))
  if not texto_transcrito:
    print("Erro: Transcrição falhou!")
    return

  prompt = f"Aqui está uma descrição de áudio: {texto_transcrito}. Com base nesta descrição e na imagem fornecida, gere um resumo textual."

  resposta_chatgpt = gerar_texto_openai(caminho_imagem, prompt)
  resposta_gemini = gerar_texto_google(caminho_imagem, prompt)

  print("Resposta ChatGPT:", resposta_chatgpt)
  print("Resposta Gemini:", resposta_gemini)

In [8]:
titulo = "Uma praia bonita. Possui coqueiros e água cristalina."
audio_gtts = tts_gtts(titulo)
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
entrada_multimodal(praia, audio_gtts)

Áudio gerado e salvo como 'gtts_audio.mp3'.
Conversão concluída: 'audio.wav'
Baixando o modelo...
Resposta ChatGPT: A imagem retrata uma praia exótica com areia branca e fina, margeada por uma vegetação exuberante de coqueiros. O mar exibe um tom azul cristalino, refletindo o céu claro e algumas nuvens brancas dispersas. Ao fundo, uma colina coberta por vegetação densa completa o cenário tropical e paradisíaco.
Resposta Gemini: Um paraíso tropical sereno se desdobra, com palmeiras inclinadas sobre uma extensão imaculada de areia branca. Águas cristalinas cintilantes beijam suavemente a costa, enquanto uma colina exuberante e coberta de vegetação se estende ao longe, criando um pano de fundo pitoresco para este oásis idílico à beira-mar. O céu azul brilhante, salpicado de nuvens fofas, completa a cena encantadora.


## 4. Transferência de modalidade
Fornecer apenas a imagem e pedir ao modelo para gerar um áudio descritivo.

In [9]:
# Transferência de modalidade
def transferencia_modalidade(imagem_path):
  prompt = "Descreva esta imagem minuciosamente para que esta possa ser convertida em formato de áudio"

  print("Gerando descrição do ChatGPT...")
  descricao_chatgpt = gerar_texto_openai(imagem_path, prompt)

  print("Gerando descrição do Gemini...")
  descricao_gemini = gerar_texto_google(imagem_path, prompt)

  # Geração do áudio do ChatGPT a partir da descrição
  if descricao_chatgpt and not descricao_chatgpt.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo ChatGPT: \n{descricao_chatgpt}")
    audio_chatgpt = tts_openai(descricao_chatgpt)
    if audio_chatgpt:
      with open("descricao_chatgpt.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_chatgpt)
      print("Áudio salvo como 'descricao_chatgpt.mp3'")
      display(Audio("descricao_chatgpt.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_chatgpt}")

  # Geração do áudio do Gemini a partir da descrição
  if descricao_gemini and not descricao_gemini.startswith("Erro"):
    print(f"Descrição gerada para áudio pelo Gemini: \n{descricao_gemini}")
    audio_gemini = tts_openai(descricao_gemini)
    if audio_gemini:
      with open("descricao_gemini.mp3", "wb") as f:
        # Salvando o áudio
        f.write(audio_gemini)
      print("Áudio salvo como 'descricao_gemini.mp3'")
      display(Audio("descricao_gemini.mp3", autoplay=True))
  else:
    print(f"Erro ao gerar descrição: {descricao_gemini}")

In [13]:
praia = '/content/drive/MyDrive/Inteligência Artificial/praia.jpg'
transferencia_modalidade(praia)

Gerando descrição do ChatGPT...
Gerando descrição do Gemini...
Descrição gerada para áudio pelo ChatGPT: 
A imagem retrata uma praia paradisíaca com areia clara e fina. À esquerda, há uma série de palmeiras inclinadas em direção ao mar, com folhas verdes vibrantes, criando sombras no chão. O mar possui um tom azul-turquesa, com pequenas ondas que se quebram suavemente na margem. Ao fundo, à direita da imagem, avista-se uma formação montanhosa coberta por vegetação verdejante, que se projeta em direção ao oceano. O céu está azul com algumas nuvens brancas dispersas, refletindo um dia claro e ensolarado. A cena transmite tranquilidade e beleza natural.
Áudio salvo como 'descricao_chatgpt.mp3'


Descrição gerada para áudio pelo Gemini: 
A imagem apresenta uma cena de praia tropical idílica. Em primeiro plano, coqueiros inclinam-se em direção à água azul-turquesa, seus troncos grossos e folhas verdes vibrantes dominando a composição. A areia é de um branco imaculado e parece macia e pulverulenta.

Em segundo plano, uma encosta exuberante e verde sobe do oceano, coberta por vegetação tropical.  Um promontório rochoso pode ser visto projetando-se para o mar à distância.

O oceano é uma mistura hipnotizante de azuis, variando de turquesa claro perto da costa até azul royal mais profundo no horizonte. Ondas suaves quebram na praia, criando uma margem branca de espuma.

O céu está claro e azul, pontilhado de nuvens brancas fofas, adicionando à beleza serena da cena. A imagem geral evoca uma sensação de tranquilidade, beleza tropical e o paraíso perfeito para férias na praia.
Áudio salvo como 'descricao_gemini.mp3'


## 5. Avaliação do Contexto

Fornecer uma sequência de imagens com descrições em áudio e verificar se o modelo consegue identificar a progressão da narrativa.

In [11]:
# Avaliação do contexto para OpenAI ChatGPT
def avaliacao_do_contexto_openai(pares_imagem_audio):
  """pares_imagem_audio: lista de tuplas (imagem_path, audio_path)
  """
  textos_transcritos = []
  imagens_bytes = []

  print("Transcrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_imagem_audio):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, audio))
      try:
        with open(imagem, "rb") as imagem_file:
          imagens_bytes.append(imagem_file.read())

        print(f"[{idx + 1}] Texto transcrito: {texto}")

      except Exception as e:
        print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
        return
    else:
      print(f"[{[idx + 1]}] Falha ao transcrever o áudio: {audio}")
      return

  print("\n Analisando progressão narrativa com ChatGPT")

  # Mensagem inicial do prompt
  mensagens = [{"role": "system", "content": "Você é um especialista em análise narrativa. Avalie se há progressão lógica entre as imagens e as descrições fornecidas."}]

  # Montar a sequência multimodal para enviar ao modelo
  for idx, (imagem_bytes, texto) in enumerate(zip(imagens_bytes, textos_transcritos)):
    mensagens.append({
        "role": "user",
        "content": [
            {"type": "text", "text": f"Imagem {idx + 1}: {texto}"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64.b64encode(imagem_bytes).decode('utf-8')}"}}
        ]
    })

  # Pergunta final
  mensagens.append({
      "role": "user",
      "content": "Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?"
  })

  try:
    # Completion para o prompt
    response = client.chat.completions.create(
        model = "gpt-4o",
        messages=mensagens
    )
    resultado = response.choices[0].message.content
    print("Avaliação do contexto (ChatGPT):")
    print(resultado)
    return resultado

  except Exception as e:
    print(f"Erro ao analisar a narrativa: {e}")
    return None

# Avaliação do contexto para Google Gemini
def avaliacao_do_contexto_google(pares_imagem_audio):
  """pares_imagem_audio: lista de tuplas (imagem_path, audio_path)
  """
  textos_transcritos = []
  imagens_bytes = []

  print("Transcrevendo áudios e carregando imagens...")
  for idx, (imagem, audio) in enumerate(pares_imagem_audio):
    texto = transcrever_audio(audio)
    if texto:
      textos_transcritos.append((imagem, audio))
      try:
        with open(imagem, "rb") as imagem_file:
          imagens_bytes.append(imagem_file.read())

        print(f"[{idx + 1}] Texto transcrito: {texto}")

      except Exception as e:
        print(f"[{idx + 1}] Erro ao ler a imagem: {e}")
        return
    else:
      print(f"[{[idx + 1]}] Falha ao transcrever o áudio: {audio}")
      return

  print("\n Analisando progressão narrativa com Gemini")

  try:
    # Carrega o modelo do Google Gemini
    model = genai.GenerativeModel("gemini-1.5-pro")

  except Exception as e:
    print(f"Erro ao inicializar o modelo Google Gemini: {str(e)}")
    return

  # Monta a entrada multimodal
  partes_entrada = [
      "Você é um especialista em análise narrativa. Avalie a sequência de imagens e descrições fornecidas.",
      "Determine se existe uma progressão narrativa lógica entre os eventos.\n"
  ]

  # Coloca os pares (imagem, áudio) no prompt
  for idx, (imagem_bytes, texto) in enumerate(zip(imagens_bytes, textos_transcritos)):
    partes_entrada.append(f"Imagem {idx + 1}: {texto}")
    partes_entrada.append({"mime_type": "image/jpeg", "data": imagem_bytes})

  # Pergunta final
  partes_entrada.append("Com base nas imagens e nas descrições anteriores, existe uma narrativa progressiva?")

  try:
    # Completion para o modelo
    resultado = model.generate_content(partes_entrada)
    print("Avaliação do contexto (Gemini):")
    print(resultado.text)
    return resultado.text

  except Exception as e:
    print(f"Erro ao gerar análise com Gemini: {e}")
    return None

In [12]:
# Carregando as imagens e áudios do Google Drive
pasta_imagens = '/content/drive/MyDrive/Inteligência Artificial/Imagens_narrativa'
pasta_audios = '/content/drive/MyDrive/Inteligência Artificial/Audios_narrativa'
pares_imagens_audios = gerar_pares_imagem_audio(pasta_imagens, pasta_audios)

avaliacao_do_contexto_openai(pares_imagens_audios)
avaliacao_do_contexto_google(pares_imagens_audios)

Conversão concluída: 'audio_convertido_1.wav'
Conversão concluída: 'audio_convertido_2.wav'
Conversão concluída: 'audio_convertido_3.wav'
Conversão concluída: 'audio_convertido_4.wav'
Conversão concluída: 'audio_convertido_5.wav'
Transcrevendo áudios e carregando imagens...
[1] Texto transcrito: o gato tobias está explorando o bairro
[2] Texto transcrito: o gato tobias acham guarda chuva do
[3] Texto transcrito: o guarda chuva se fecha e abrir um portal
[4] Texto transcrito: o gato tobias sugado pelo portal
[5] Texto transcrito: o gato tobias chega em um bibelô polícia recebido pelo coelho estiloso

 Analisando progressão narrativa com ChatGPT
Avaliação do contexto (ChatGPT):
Sim, há uma narrativa progressiva nas imagens e descrições fornecidas:

1. **Imagem 1**: Aparece um guarda-chuva azul diante de um gato curioso, sugerindo que um portal surge quando o guarda-chuva se fecha.
2. **Imagem 2**: O gato, chamado Tobias, encontra o guarda-chuva azul, possivelmente dando início à aventura

'Não, a sequência de imagens e descrições fornecidas **não apresenta uma progressão narrativa lógica**. Há elementos que sugerem uma história, mas a ordem e as ações não constroem um enredo coerente. Vamos analisar por partes:\n\n* **Imagem 1 e 2:**  A primeira imagem mostra um gato encontrando um guarda-chuva fechado. Na segunda, o gato está azul, sentado perto do guarda-chuva aberto. A mudança de cor sugere uma transformação relacionada ao objeto, mas não há uma transição clara.  Falta uma etapa que explique a mudança de cor.\n\n* **Imagem 3:**  Aqui, o gato (de volta à cor original) entra num portal azul, aparentemente ligado ao guarda-chuva, e chega a uma cidade chamada Umbrelopolis. Esta imagem se conecta logicamente à ideia do guarda-chuva mágico introduzida nas imagens anteriores, apesar da inconsistência da cor do gato.\n\n* **Imagem 4:** O gato está explorando o que parece ser um bairro comum.  Esta imagem quebra a sequência lógica.  Após entrar num portal para uma cidade temá